## Analyze Accuracy per class

In [ ]:
import os
import warnings

import pandas as pd
from PIL import Image, ImageDraw, ImageFont

warnings.filterwarnings("ignore")

### topex-printer (RGB)

In [ ]:
ds_source = "../data/topex-printer/train"
ds_target = "../data/topex-printer/test"
df_classacc = pd.read_csv(
    "../data/eval-data/STL-topex--uda--swinv2--ch-uda_cdan-mcc_adamw1e-5_warmupcalr_augmix_classacc.csv"
)
df_confmat = pd.read_csv(
    "../data/eval-data/STL-topex--uda--swinv2--ch-uda_cdan-mcc_adamw1e-5_warmupcalr_augmix_confmat.csv"
)

In [ ]:
display(df_classacc.head())
display(df_classacc.describe())
display(df_confmat.head())

In [ ]:
def count_files(dir):
    return len([name for name in os.listdir(dir) if os.path.isfile(os.path.join(dir, name))])

In [ ]:
IMSIZE = 100
SPACER = 10
TEXTSIZE = 20

df_sorted = df_classacc.sort_values(by="acc", ascending=False)
for cls in df_sorted["class_name"]:
    n_source = count_files(f"{ds_source}/{cls}")
    n_target = count_files(f"{ds_target}/{cls}")
    acc_target = df_sorted[df_sorted["class_name"] == cls]["acc"].values[0]
    print("----------------------------------------")
    print(f"{cls}")

    cls_conf = df_confmat[
        (df_confmat["Actual"] == cls) & (df_confmat["nPredictions"] > 0) & (df_confmat["Predicted"] != cls)
    ]
    n_imgs = len(cls_conf) + 1
    img_conf_grid = Image.new("RGBA", ((IMSIZE + SPACER) * n_imgs, IMSIZE * 2 + SPACER + TEXTSIZE * 2))

    img_source = Image.open(f"{ds_source}/{cls}/train_{cls}_0001_0010.png").resize((IMSIZE, IMSIZE))
    img_target = Image.open(f"{ds_target}/{cls}/test_{cls}_0001.png").resize((IMSIZE, IMSIZE))
    img_conf_grid.paste(img_source, (0, TEXTSIZE))
    img_conf_grid.paste(img_target, (0, TEXTSIZE + IMSIZE + SPACER))

    n_target_pred = df_confmat[(df_confmat["Actual"] == cls) & (df_confmat["Predicted"] == cls)]["nPredictions"].values[
        0
    ]
    font = ImageFont.load_default()
    text = f"{n_target_pred}/{n_target} | {acc_target:.2f}"
    text_color = (0, 0, 0)
    text_width, text_height = draw.textsize(text, font=font)
    text_position = (0, TEXTSIZE + IMSIZE * 2 + SPACER)
    draw = ImageDraw.Draw(img_conf_grid)
    draw.text(text_position, text, font=font, fill=text_color)
    text = cls
    text_position = (0, 0)
    draw.text(text_position, text, font=font, fill=text_color)

    for i, pred_cls in enumerate(cls_conf["Predicted"]):
        conf_n_pred = cls_conf[(cls_conf["Predicted"] == pred_cls)]["nPredictions"].values[0]
        conf_ratio = conf_n_pred / n_target
        img_conf_source = Image.open(f"{ds_source}/{pred_cls}/train_{pred_cls}_0001_0010.png").resize((IMSIZE, IMSIZE))
        img_conf_grid.paste(img_conf_source, ((i + 1) * (IMSIZE + SPACER), TEXTSIZE))
        img_conf_target = Image.open(f"{ds_target}/{pred_cls}/test_{pred_cls}_0001.png").resize((IMSIZE, IMSIZE))
        img_conf_grid.paste(img_conf_target, ((i + 1) * (IMSIZE + SPACER), TEXTSIZE + IMSIZE + SPACER))

        text = f"{conf_n_pred}/{n_target} | {conf_ratio:.2f}"
        if conf_ratio < 0.05:
            text_color = (0, 255, 0)
        elif conf_ratio < 0.2:
            text_color = (255, 255, 0)
        else:
            text_color = (255, 0, 0)
        text_width, text_height = draw.textsize(text, font=font)
        text_position = ((i + 1) * (IMSIZE + SPACER), TEXTSIZE + IMSIZE * 2 + SPACER)
        draw.text(text_position, text, font=font, fill=text_color)

    display(img_conf_grid)